In [ ]:
# Tomado de: https://www.analyticsvidhya.com/blog/2020/10/multivariate-multi-step-time-series-forecasting-using-stacked-lstm-sequence-to-sequence-autoencoder-in-tensorflow-2-0-keras/
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns 
import networkx as nx
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data = pd.read_csv('/content/gdrive/Shareddrives/Network Science/PROYECTO/Serie de tiempo/serie.csv', sep=',')
data

In [ ]:
data.head()

In [ ]:
data['Distrito'] = pd.to_numeric(data['Distrito'])

In [ ]:
data.info()

In [ ]:
# Conversion de los datos a numpy array
valores = data.values

# Construcion de escalador
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(valores)
print(scaler)

# Escalamiento de los valores
normalizados = scaler.transform(valores)
df_normalizados = pd.DataFrame(normalizados,
                              index=data.index,
                              columns=data.columns)
df_normalizados

In [ ]:
data.info()
y = np.array(df_normalizados['Distrito'])
X = np.array(df_normalizados[['Source','Target','Time']])

In [ ]:
df_normalizados.head()

In [ ]:
G = nx.from_pandas_edgelist(data, source='Source', target='Target', edge_attr=["Distrito","Time"])

In [ ]:
nx.convert_node_labels_to_integers(G)
degree_centrality = nx.degree_centrality(G)  
sorted(degree_centrality.items(), key=lambda x:x[1], reverse = True)[:3]

In [ ]:
closeness_centrality = nx.closeness_centrality(G)  
sorted(closeness_centrality.items(), key=lambda x:x[1], reverse = True)[:3]

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G) 
sorted(betweenness_centrality.items(), key=lambda x:x[1], reverse = True)[:3]

In [ ]:
daily_df = df_normalizados

In [ ]:
train_number = int (len(df_normalizados)*0.7)
train_df,test_df = daily_df[0:train_number], daily_df[train_number+1:len(df_normalizados)] 
train_number

In [ ]:
len(test_df)

In [ ]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


In [ ]:
n_past = 10
n_future = 2
n_features = 4

In [ ]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], n_past, -1))
y_train = y_train.reshape((y_train.shape[0], n_future, -1))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], n_past, -1))
y_test = y_test.reshape((y_test.shape[0], n_future, -1))


In [ ]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

In [ ]:
import datetime
log_dir = "/content/gdrive/Shareddrives/Network Science/PROYECTO/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_e1d1.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss='categorical_crossentropy',
    weighted_metrics=["acc"]
)

In [ ]:
history_e1d1 = model_e1d1.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test,y_test),
    batch_size=32,
    verbose=0,
    callbacks=[tensorboard_callback]
    # callbacks=[EarlyStopping(monitor='acc', patience=20)],
)

In [ ]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e1d1

In [ ]:
%load_ext tensorboard

%tensorboard --logdir logs/fit

In [ ]:
pred_e1d1=model_e1d1.predict(X_test)

    
len(pred_e1d1)
pred_e1d1




In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.Huber()

model_e1d2.compile(
    optimizer=optimizer, 
    loss=loss_fn,
    weighted_metrics=["acc"]
)
history_e1d1 = model_e1d2.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_test,y_test),
    batch_size=32,
    verbose=0,
    callbacks=[EarlyStopping(monitor='acc', patience=20)],
)

In [ ]:
def display_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history["loss"])
    ax1.plot(history.history["val_loss"])
    ax1.legend(["train", "test"], loc="upper right")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")

    ax2.plot(history.history["acc"])
    ax2.plot(history.history["val_acc"])
    ax2.legend(["train", "test"], loc="upper right")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    plt.show()

In [ ]:
display_learning_curves(history_e1d1)